In [1]:
import numpy as np
import torch
from torch import nn
import pandas as pd
from tqdm import tqdm

In [2]:
torch.cuda.is_available()

False

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

In [4]:
class TorchLinearRegression(nn.Module):
    def __init__(self):
        super(TorchLinearRegression, self).__init__()
        self.W = nn.Parameter(torch.rand(2, 1), requires_grad=True)
        self.b = nn.Parameter(torch.rand(1), requires_grad=True)
#         self.linearblock = torch.nn.Linear(2, 1)
        
    def forward(self, x):
        return torch.mm(x, self.W) + self.b
#         return self.linearblock(x)

In [5]:
dataset = pd.read_csv('dataset1.csv')
dataset.head()

,x1,x2,y
0,0.0,0.0,1.499610
1,0.0,0.1,1.350266
2,0.0,0.2,1.185698
3,0.0,0.3,1.055393
4,0.0,0.4,0.900658


In [15]:
data = torch.tensor([dataset['x1'], 
                     dataset['x2'],
                     dataset['y']]).to(device)
data = torch.t(data)
dataloader = torch.utils.data.DataLoader(data)

# alternative: using Dataset (https://stanford.edu/~shervine/blog/pytorch-how-to-generate-data-parallel)

In [16]:
# training
model = TorchLinearRegression()

criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters())

In [20]:
nb_epochs = 10

for _ in tqdm(range(nb_epochs)):
    for i, data in enumerate(dataloader):
        optimizer.zero_grad()
        y_test = torch.tensor([data[:, -1]])
        y_pred = model(data[:, :-1])
        if i == 1:
            print(y_test)
            print(y_pred)
        loss = criterion(y_pred, y_test)
        loss.backward()
        optimizer.step()





  0%|          | 0/10 [00:00<?, ?it/s]

tensor([1.3503])
tensor([[1.4215]], grad_fn=<AddBackward0>)


KeyboardInterrupt: 

In [15]:
torch.save(model.state_dict(), 'linear_regression_model.pkl')

In [18]:
model1 = TorchLinearRegression()
model1.load_state_dict(torch.load('linear_regression_model.pkl'))

<All keys matched successfully>

In [19]:
model1(torch.tensor([[1., 2.]]))

tensor([[1.3296]], grad_fn=<AddBackward0>)

In [25]:
model(torch.tensor([[1., 2.]]))

tensor([[1.3296]], grad_fn=<AddBackward0>)